In [ ]:
!unzip -q /content/drive/MyDrive/Project_4_export.zip

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 24.8 MB/s eta 0:00:00


# 1. Long-Short Term Memory (LSTM) for Sentiment Analysis (20 points)
Your task is to create an LSTM network to predict sentiment from Yelp reviews (train_yelp_reviews.csv).
A review is considered positive if labeled with a `1` and negative if labeled with a `0`. (Hint: Read
the provided CSV files with pandas using sep =`\t` and engine=`python`)

You may use any packages or tools as you wish, however, the code that you submit must be
written on your own. You are free to experiment with pre-processing, model architectures, training
procedures, removing stop-words, or hyper-parameters, as long as your model contains at least one
LSTM layer. You may find using a GPU to be beneficial, but we have made sure that good classification
can be obtained using an average CPU.

## 1.1 Data Inspection (5 points)
Read through some of the reviews. Display the most 50 commonly occurring tokens for each class.
Are there any patterns in reviews that Naive Bayes or other bag-of-word models may not be able
to classify accurately?

In [ ]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import re
warnings.filterwarnings('ignore') 
from numpy.random import seed
seed(10)

import os
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import LabelEncoder

In [ ]:
train_data = pd.read_csv('/content/datasets/train_yelp_reviews.csv', sep='\t', engine = 'python')
train_data.head(5)

,text,label
0,Great time - family dinner on a Sunday night.,1
1,The classic Maine Lobster Roll was fantastic.,1
2,We won't be going back.,0
3,All I have to say is the food was amazing!!!,1
4,"Food was good, service was good, Prices were g...",1


In [ ]:
#positive reviews and negative reviews
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from nltk.probability import FreqDist
import re

nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

punctuations = set(string.punctuation)

positive_reviews = []
negative_reviews = []

## implementation understanding taken from chatgpt
for idx, row in train_data.iterrows():
  tokens = word_tokenize(row['text'])  
  tokens = [token.lower() for token in tokens if token.lower() not in stop_words and token not in punctuations]


  if row['label'] == 1:
    positive_reviews.extend(tokens)

  if row['label'] == 0:
    negative_reviews.extend(tokens)

## referenced from https://tedboy.github.io/nlps/generated/generated/nltk.FreqDist.html
positive_reviews_dict = FreqDist(positive_reviews)
negative_reviews_dict = FreqDist(negative_reviews)

positive_reviews_dict.most_common(50)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


[('good', 66),
 ('great', 64),
 ('food', 53),
 ('place', 51),
 ('service', 42),
 ('friendly', 22),
 ('delicious', 21),
 ('amazing', 20),
 ('back', 20),
 ('really', 20),
 ('best', 20),
 ('nice', 19),
 ('time', 18),
 ("'s", 18),
 ("n't", 17),
 ('go', 17),
 ('also', 16),
 ('like', 16),
 ('restaurant', 15),
 ('staff', 13),
 ('...', 13),
 ('fantastic', 12),
 ('always', 12),
 ('vegas', 12),
 ('love', 12),
 ('awesome', 12),
 ('pretty', 11),
 ('menu', 11),
 ('first', 10),
 ('fresh', 10),
 ('excellent', 10),
 ('steak', 10),
 ('pizza', 10),
 ('experience', 10),
 ('even', 10),
 ('perfect', 9),
 ('chicken', 9),
 ('atmosphere', 9),
 ('server', 9),
 ('one', 9),
 ('prices', 8),
 ('stars', 8),
 ('everything', 8),
 ('made', 8),
 ('spot', 8),
 ('loved', 8),
 ('definitely', 8),
 ('ever', 8),
 ('well', 8),
 ('come', 8)]

In [ ]:
negative_reviews_dict.most_common(50)

[("n't", 68),
 ('food', 60),
 ('place', 40),
 ('back', 33),
 ('service', 31),
 ('like', 28),
 ('go', 25),
 ('would', 20),
 ('good', 20),
 ('time', 19),
 ("'s", 18),
 ('never', 17),
 ('bad', 16),
 ('minutes', 16),
 ('ever', 16),
 ('one', 15),
 ('...', 14),
 ("'ve", 14),
 ('disappointed', 14),
 ('got', 13),
 ('much', 12),
 ('think', 12),
 ('us', 12),
 ('really', 12),
 ("'m", 12),
 ('came', 12),
 ('wo', 11),
 ('going', 11),
 ('worst', 11),
 ('get', 10),
 ('better', 10),
 ('eat', 10),
 ('probably', 9),
 ('even', 9),
 ('``', 9),
 ("''", 9),
 ('bland', 9),
 ('slow', 9),
 ('wait', 9),
 ('terrible', 9),
 ('waited', 9),
 ('also', 8),
 ('burger', 8),
 ('way', 8),
 ('flavor', 8),
 ('restaurant', 8),
 ('experience', 8),
 ('coming', 8),
 ('ordered', 8),
 ('another', 8)]

The patterns in reviews that Naive Bayes or other bag-of-word models can find difficult to categorize correctly are frequently connected to the meaning of words in context. Bag-of-word models do not take into account the order or sequence of words in a sentence and instead treat each word as an independent property. 

## 1.2 Model Training (15 points)
Implement your LSTM model and apply it to the training dataset. In order to obtain full credit, you
must describe your text pre-processing procedure, describe your network in terms of the layers used,
input/output dimensions at each layer, choice of word embedding dictionary, and training procedure.

In [ ]:
from sklearn.model_selection import train_test_split

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['text'])
sequences = tokenizer.texts_to_sequences(train_data['text'])

# Pad sequences to ensure equal length
max_len_seq = 0
for seq in sequences:
  len_seq = len(seq)
  if max_len_seq < len_seq:
    max_len_seq = len_seq
    
padded_sequences = pad_sequences(sequences, maxlen=max_len_seq)

# Split the data into training and validation sets
xtrain, xval, ytrain, yval = train_test_split(padded_sequences, train_data['label'], test_size=0.2, random_state=42)

xtrain.shape

(720, 32)

In [ ]:
##LSTM
model = Sequential()
model.add(tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=200, input_length=max_len_seq))
model.add(tf.keras.layers.SpatialDropout1D(0.4))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)))
model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.SpatialDropout1D(0.3))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(Dense(1,activation='sigmoid'))


reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, cooldown=3, mode = 'min')
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
loss = tf.keras.losses.BinaryCrossentropy( 
    from_logits = True
)

metric = tf.keras.metrics.BinaryAccuracy(
    name='accuracy',
)

model.compile(loss=loss,
              optimizer='adam',
              metrics=[metric])

In [ ]:
from sklearn.metrics import *

num_epochs = 10
history = model.fit(xtrain, ytrain, 
                    epochs=num_epochs, verbose=1,
                    validation_data = (xval, yval),
                    batch_size = 32,
                    callbacks = [reduce_lr, early])

prediction = model.predict(xval)

# Get labels based on probability 1 if p>= 0.5 else 0
pred_labels = []
for i in prediction:
    if i >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)
print("Report of prediction on test set : \n ", classification_report(yval, pred_labels))

Epoch 1/10
23/23 [==============================] - 19s 239ms/step - loss: 0.6956 - accuracy: 0.5347 - val_loss: 0.6922 - val_accuracy: 0.5056 - lr: 0.0010
Epoch 2/10
23/23 [==============================] - 3s 133ms/step - loss: 0.6820 - accuracy: 0.5542 - val_loss: 0.6925 - val_accuracy: 0.5056 - lr: 0.0010
Epoch 3/10
23/23 [==============================] - 4s 168ms/step - loss: 0.6014 - accuracy: 0.6861 - val_loss: 0.6811 - val_accuracy: 0.5167 - lr: 0.0010
Epoch 4/10
23/23 [==============================] - 3s 122ms/step - loss: 0.3343 - accuracy: 0.8681 - val_loss: 0.6244 - val_accuracy: 0.7722 - lr: 0.0010
Epoch 5/10
23/23 [==============================] - 2s 72ms/step - loss: 0.1543 - accuracy: 0.9403 - val_loss: 0.5825 - val_accuracy: 0.7389 - lr: 0.0010
Epoch 6/10
23/23 [==============================] - 2s 78ms/step - loss: 0.1163 - accuracy: 0.9611 - val_loss: 0.5451 - val_accuracy: 0.7167 - lr: 0.0010
Epoch 7/10
23/23 [==============================] - 1s 38ms/step - loss

In [ ]:
print("F1 Score : {}".format(f1_score(yval, pred_labels)))

F1 Score : 0.7669902912621359


In [ ]:
print("Confusion Matrix : \n {}".format(confusion_matrix(yval, pred_labels)))

Confusion Matrix : 
 [[53 36]
 [12 79]]


## 1.3 Model Prediction
Apply your trained model to the test data (test_yelp_reviews.csv). Save your predictions as a
new file in single column titled "prediction" and export as a CSV file. Please submit your saved
CSV to Canvas with the name (<YourNetID>_LSTM_predictions.csv). Your grade on the model
will be based on the accuracy of your predictions on the unlabeled data. Below is an example
output:

In [ ]:
stop_words = set(stopwords.words('english'))
test_data = pd.read_csv('/content/datasets/test_yelp_reviews.csv', sep='\t',engine='python')

# test_data['text'] = test_data['text'].apply(lambda x: [token.lower() for token in tokens if token.lower() not in stop_words and token not in punctuations])
# test_data['text'] = test_data['text'].apply(lambda x: ' '.join(x))

# # Convert the text data to sequences or embeddings if required by your model
test_sequences = tokenizer.texts_to_sequences(test_data['text'])

# # Pad sequences to a fixed length
test_sequences = pad_sequences(test_sequences, maxlen=max_len_seq)

# Make predictions
test_preds = model.predict(test_sequences)
test_preds = test_preds.flatten()

pred_labels = []
for i in test_preds:
    if i >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)
# Create a DataFrame with predictions
preds_df = pd.DataFrame({'predictions': pred_labels})

# Save predictions to a CSV file
preds_df.to_csv('/content/drive/MyDrive/f006d3c_LSTM_predictions.csv', index=False)

4/4 [==============================] - 0s 8ms/step


# 2. FineTune Bert Model (20 points)
Using the same dataset, fine-tune a pre-trained BERT model for sentiment analysis. You are
recommended to use the Hugging Face implementation of BERT model and pre-trained weights.

• Preprocess the dataset into a format that can be used by BERT model.

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification, BertModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from numpy.random import seed
seed(100)

xtrain, xtest = train_test_split(train_data, test_size=0.2, random_state=45)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the text and encode labels
train_encodings = tokenizer(list(xtrain['text']), truncation=True, padding=True)
val_encodings = tokenizer(list(xtest['text']), truncation=True, padding=True)

# Create TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    xtrain['label'].values
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    xtest['label'].values
))

Fine-tune the pre-trained BERT model using the training set and evaluate its performance on
the validation dataset.

In [ ]:
# Load the pre-trained BERT model
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

# Define the optimizer, loss function, and metrics optimizer, and loss suggested by chatgpt
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

#asked chatgpt about recommended metrics for BERT, and it suggested this
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# callbacks = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=5)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, cooldown=3, mode = 'min')
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# Compile the model
bert_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
#bert_model.compile(optimizer=optimizer, loss=loss, metrics='accuracy')

# Train the model
bert_model.fit(
    train_dataset.shuffle(1000).batch(64),
    epochs=15,
    batch_size=32,
    validation_data=val_dataset.shuffle(1000).batch(64),
    callbacks = [reduce_lr, early]
)


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/15
12/12 [==============================] - 63s 858ms/step - loss: 0.6575 - accuracy: 0.6347 - val_loss: 0.5395 - val_accuracy: 0.7611 - lr: 2.0000e-05
Epoch 2/15
12/12 [==============================] - 6s 534ms/step - loss: 0.3973 - accuracy: 0.9000 - val_loss: 0.3032 - val_accuracy: 0.9111 - lr: 2.0000e-05
Epoch 3/15
12/12 [==============================] - 7s 566ms/step - loss: 0.2237 - accuracy: 0.9431 - val_loss: 0.3152 - val_accuracy: 0.8944 - lr: 2.0000e-05
Epoch 4/15
12/12 [==============================] - 6s 539ms/step - loss: 0.1476 - accuracy: 0.9653 - val_loss: 0.2274 - val_accuracy: 0.9222 - lr: 2.0000e-05
Epoch 5/15
12/12 [==============================] - 7s 569ms/step - loss: 0.0720 - accuracy: 0.9889 - val_loss: 0.2371 - val_accuracy: 0.9222 - lr: 2.0000e-05
Epoch 6/15
12/12 [==============================] - 6s 529ms/step - loss: 0.0500 - accuracy: 0.9931 - val_loss: 0.2462 - val_accuracy: 0.9167 - lr: 2.0000e-05
Epoch 7/15
12/12 [===========================

Compare the performance of the BERT model with the LSTM model and comment on your
findings.

Based on the initial analysis, I found that BERT model is performing much better than the LSTM model, the F1 Score of LSTM model was 0.785, however for BERT it was 0.92.

In [ ]:
val_predictions = bert_model.predict(val_dataset.batch(64))
val_pred_labels = np.argmax(val_predictions.logits, axis=1)

# Calculate classification report
print(classification_report(xtest['label'].values, val_pred_labels))

3/3 [==============================] - 4s 146ms/step
              precision    recall  f1-score   support

           0       0.93      0.91      0.92        90
           1       0.91      0.93      0.92        90

    accuracy                           0.92       180
   macro avg       0.92      0.92      0.92       180
weighted avg       0.92      0.92      0.92       180



In [ ]:
print("F1 Score : {}".format(f1_score(xtest['label'].values, val_pred_labels)))

F1 Score : 0.9230769230769231


Apply your trained model to the test data (test_yelp_reviews.csv). Save your predictions
as a new file in single column titled "classification" and export as a CSV file.
Please submit a copy of your saved CSV as a separate file on Canvas with the format
(<YourNetID>_BERT_predictions.csv). Your grade on the model will be based on the
F1-score of your predictions on the unlabeled data.

In [ ]:
test_data = pd.read_csv('/content/datasets/test_yelp_reviews.csv', sep='\t',engine='python')

# Convert the text data to sequences or embeddings if required by your model
test_sequences = tokenizer(list(test_data['text']), truncation=True, padding=True)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_sequences)
))

# Make predictions
test_preds = bert_model.predict(test_dataset.batch(64))
# Create a DataFrame with predictions
test_preds = np.argmax(test_preds.logits, axis=1)

preds_df = pd.DataFrame({'predictions': test_preds})

# Save predictions to a CSV file
preds_df.to_csv('/content/drive/MyDrive/f006d3c_BERT_predictions.csv', index=False)

2/2 [==============================] - 4s 135ms/step


# 4. Large Language Models (30 points)
In this part, you will be asked several questions about large language models. No coding is needed.
6 points for each question.

• Why is it challenging to train language models using reinforcement learning with human feedback (RLHF)? Give three examples to demonstrate.

It is challenging to train language models using reinforcement learning with human feedback, as there are predominantly 3 issues that would occur if we were to follow these steps.

1. Lack of Scalability as the RLHF-based modeling approach would require a large dataset consisting of human-generated training datasets, this can be time intensive and computationally expensive process.
2. Delayed sparse rewards that are commonly received by the language models as feedback signals would make it difficult for the model to learn from its processes to receive rewards making it a slower and unstable process.
3. Trade-off between exploration and exploitation: In RLHF, it's critical to strike a balance between the two. Language models must investigate various actions to find superior tactics while making use of previously acquired knowledge. It can be difficult to strike the ideal balance between exploration and exploitation.

• What are pros and cons of instruction finetuning and RLHF?

Instruction Finetuning is very much interpretable as it makes decision based on the inputs provided by us providing us with precise control on how we would want to see the output of the model.

One of the drawbacks for this model is that it is a time consuming and laborious process to provide the model with instructions. It also lacks generalizability and that it would be required to be specifically trained for those scenarios. 

RLHF on the other hand can be generalizable and would be able to adapt to different scenarios as it learns from feedbacks provided by us. The major drawbacks for RLHF would be that it is very expensive to train the model due to large number of parameters required to train the model.

• What are some of the challenges of evaluating the current generation of language models?

1. One of the main concerns and issues that plague the current generation of language models is the data quality and diversity of the dataset the model is being trained on. How well the data has been collected affects the model performance, poor quality will often lead the model to overfitting, bias and have generalizibility issues. 

2. Ethical and social implications is another area of concern that can potentially validate the current generation of language models. These models can impact society, as it can generate misleading or unethical contents. It can also empower people by improving how people communicate with each others, and gain information.

3. Selecting adequate evaluation metrics and criteria is the third difficulty that has an impact on language models. The effectiveness and quality of the language models can be measured through evaluation measures. Conventional evaluation metrics, however, might not be adequate or appropriate for all models. For instance, certain metrics may ignore deeper-level factors like coherence, relevance, logic, and creativity in favor of focusing exclusively on the surface-level accuracy or fluency of the created content. Additionally, some criteria could be ambiguous.

• Compare three popular language models (you can choose ChatGPT and any other two, e.g., https://www.perplexity.ai/, https://you.com/, https://nat.dev/, https://bard.google.com/). Come up with 5 challenging questions, and compare their output. Comment on the quality of the output and explain what is the possibly the reason.

I asked them about various challenging questions focusing on the realm of quantum computing, to applications of Natural language processing in resolving bias and eradicating iliteracy to whether god exists or not. What I found out that the response was similar for the Chatgpt, Perplexity, and you with changing in the way they phrased the sentences. The quality of output was good and it seems that though it seems complicated they were fairly able to avoid any bias or any kind of ethical issues with their responses.



Reference : https://www.linkedin.com/advice/1/what-some-current-challenges-limitations-language